<a href="https://colab.research.google.com/github/Aniket-desai-24/Machine-Learning/blob/main/Sentiment%20Analysis%20of%20Extracted%20Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Text Analysis: Sentiment Analysis of Extracted Articles**


####The objective of this assignment is to extract textual data articles from a given URL and perform text analysis to compute variables. The process involves data extraction and data analysis ,explain methodology adopted to perform text analysis to drive sentimental opinion, sentiment scores, readability, passive words, personal pronouns and etc.

In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Tasks/20211030 Test Assignment

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Tasks/20211030 Test Assignment


The provided code snippet imports libraries such as requests, BeautifulSoup, pandas, nltk, and re. These libraries are commonly used for web scraping, text analysis, and data manipulation tasks. The code also downloads NLTK's tokenization and stopwords resources. The purpose of the code seems to involve extracting data from web pages, preprocessing the text using NLTK and regular expressions, and potentially utilizing Pandas for data storage and manipulation.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The code reads URLs from an Excel file, extracts article titles and text from the web pages using BeautifulSoup, and saves them in separate text files. It handles errors during the extraction process and continues to the next URL. However, there are some URLs (ID 44, 57, 144) that couldn't be found on the website.

In [ ]:
#read the url file into the pandas object to do further opration.
df = pd.read_excel('Input.xlsx')

#loop throgh each row in the df
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']

  # make a request to url to Access tha Data.
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #create a beautifulsoup object

  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))


  #find title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue


  #find text in this most of the site have same tag for article text but some of them are in different tag so i have used both.

  article_text = ""

  all_text=soup.find('div',class_="td-post-content tagdiv-type")


  if all_text:
    try:

      for p in all_text.find_all('p'):
        article_text += p.get_text()

    except:
      print("can't get text of {}".format(url_id))
      continue

  else :
    remain_text=soup.find_all('div',class_="tdb-block-inner td-fix-index")
    remain_text=remain_text[14]
    try:

      for p in remain_text.find_all('p'):
        article_text += p.get_text()

    except:
      print("can't get text of {}".format(url_id))
      continue



  #write title and text to the file
  file_name = '/content/drive/MyDrive/Tasks/20211030 Test Assignment/TitleText/' + str(url_id) + '.txt'
  with open(file_name, 'w') as file:
    file.write(title + '\n' + article_text)

can't get title of 44
can't get title of 57
can't get title of 144


**The code snippet defines three directories:**

1)text_dir: This directory is used for storing the extracted article text files.

2)stopwords_dir: This directory is likely used for storing stop words, which are common words excluded during text analysis.

3)sentment_dir: This directory may contain a master dictionary or lexicon for sentiment analysis.
These directories are essential for organizing the output files, managing stop words, and accessing the sentiment dictionary in the provided code.

In [ ]:
# Directories
text_dir = "/content/drive/MyDrive/Tasks/20211030 Test Assignment/TitleText"
stopwords_dir = "/content/drive/MyDrive/Tasks/20211030 Test Assignment/StopWords"
sentment_dir = "/content/drive/MyDrive/Tasks/20211030 Test Assignment/MasterDictionary"

The code loads stop words from files in the stopwords_dir directory and adds them to a set called stop_words. It iterates over each file, reads its content, and updates the set by adding the stop words from each file. This ensures that the stop_words set contains a consolidated set of stop words from all the files in the directory.

In [ ]:
# Load all stop words from the stopwords directory and store in the stop_words set variable
stop_words = set()
for filename in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir, filename), 'r', encoding='ISO-8859-1') as f:
        stop_words.update(set(f.read().splitlines()))


The code loads text files, tokenizes their content, removes stop words, and stores the filtered tokens in the docs list. It performs these steps: initializes an empty list called docs, iterates over each file in the text_dir directory, reads the file content, tokenizes it, removes stop words, and appends the filtered tokens to the docs list.

In [ ]:
# Load all text files from the text directory and store in a list (docs)
docs = []
for text_file in os.listdir(text_dir):
    file_path = os.path.join(text_dir, text_file)
    if os.path.isdir(file_path):  # Skip directories
        continue
    with open(file_path, 'r') as f:
        text = f.read()
        # Tokenize the given text file
        words = word_tokenize(text)
        # Remove the stop words from the tokens
        filtered_text = [word for word in words if word.lower() not in stop_words]
        # Add each filtered token of each file into a list
        docs.append(filtered_text)


The code loads positive and negative words from files in the sentment_dir directory and stores them in separate sets, pos and neg. It reads the content of each file and adds the words to the corresponding set. The sets can be used for sentiment analysis to determine the sentiment polarity of text data.

In [ ]:
# Store positive and negative words from the directory
pos = set()
neg = set()

for files in os.listdir(sentment_dir):
    if files == 'positive-words.txt':
        with open(os.path.join(sentment_dir, files), 'r', encoding='ISO-8859-1') as f:
            pos.update(f.read().splitlines())
    else:
        with open(os.path.join(sentment_dir, files), 'r', encoding='ISO-8859-1') as f:
            neg.update(f.read().splitlines())

The code calculates sentiment-related variables for each document in the docs list. It collects positive and negative words, calculates positive and negative scores, and derives polarity and subjectivity scores. The results are stored in separate lists for further analysis.

In [ ]:
# Initialize lists for positive and negative words, scores, and polarity/subjectivity scores
positive_words = []
negative_words = []
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

# Iterate through the list of docs
for doc in docs:

    # Collect the positive and negative words from each document
    positive_words.append([word for word in doc if word.lower() in pos])
    negative_words.append([word for word in doc if word.lower() in neg])

    # Calculate the scores from the positive and negative words
    positive_score.append(len(positive_words[-1]))
    negative_score.append(len(negative_words[-1]))
    polarity_score.append((positive_score[-1] - negative_score[-1]) / ((positive_score[-1] + negative_score[-1]) + 0.000001))
    subjectivity_score.append((positive_score[-1] + negative_score[-1]) / (len(doc) + 0.000001))


The code calculates various text analysis variables for each file in the text_dir directory. It calculates the average sentence length, percentage of complex words, Fog Index, count of complex words, and average syllable count per word. The results are stored in separate lists for further analysis.

In [ ]:
avg_sentence_length = []
Percentage_of_Complex_words = []
Fog_Index = []
complex_word_count = []
avg_syllable_word_count = []

stopwords = set(stopwords.words('english'))
def measure(file):
    with open(os.path.join(text_dir, file), 'r') as f:
        text = f.read()

    # remove punctuations
    text = re.sub(r'[^\w\s.]', '', text)

    # split the given text file into sentences
    sentences = text.split('.')

    # total number of sentences in a file
    num_sentences = len(sentences)

    # total words in the file
    words = [word for word in text.split() if word.lower() not in stopwords]
    num_words = len(words)

    # complex words having syllable count is greater than 2
    complex_words = []
    for word in words:
        vowels = 'aeiou'
        syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
        if syllable_count_word > 2:
            complex_words.append(word)

    # Syllable Count Per Word
    syllable_count = 0
    syllable_words = []
    for word in words:
        if word.endswith('es'):
            word = word[:-2]
        elif word.endswith('ed'):
            word = word[:-2]
        vowels = 'aeiou'
        syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
        if syllable_count_word >= 1:
            syllable_words.append(word)
            syllable_count += syllable_count_word

    avg_sentence_len = num_words / num_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    Percent_Complex_words = len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words), avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):

    # Skip directories this is used beacuse of the .ipynb_Ckeckpoint directory
    if os.path.isdir(os.path.join(text_dir, file)):
        continue

    AS, PCW, FI, CWC, ASW = measure(file)
    avg_sentence_length.append(AS)
    Percentage_of_Complex_words.append(PCW)
    Fog_Index.append(FI)
    complex_word_count.append(CWC)
    avg_syllable_word_count.append(ASW)


The code calculates the word count and average word length for each file in the text_dir directory. It removes punctuations and stop words from the text, and then calculates the word count by counting the cleaned words and the average word length by dividing the total length of words by the number of cleaned words. The results are stored in the word_count and average_word_length lists, respectively.

In [ ]:
# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by
# removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

word_count = []
average_word_length = []

def cleaned_words(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stopwords]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length


for file in os.listdir(text_dir):
  # Skip directories this is used beacuse of the .ipynb_Ckeckpoint directory
    if os.path.isdir(os.path.join(text_dir, file)):
        continue
    x, y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)

The code counts the occurrences of specific personal pronouns ("I," "we," "my," "ours," and "us") in each file within the text_dir directory. It stores the counts in a list called pp_count for further analysis.

In [ ]:
# To calculate Personal Pronouns mentioned in the text, we use regex to find
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.


def count_personal_pronouns(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

pp_count = []
for file in os.listdir(text_dir):
  if os.path.isdir(os.path.join(text_dir, file)):
        continue
  x = count_personal_pronouns(file)
  pp_count.append(x)

The code reads an Excel file called "Output Data Structure.xlsx" into a DataFrame named output_df. It drops specific rows from the DataFrame where the URL_ID corresponds to non-existing pages.

The code then defines a list of variables that include the calculated analysis parameters. It iterates over each variable and assigns its values to the corresponding column in the output_df DataFrame.

Finally, the updated DataFrame is saved as a CSV file named "Output_Data.csv".

In summary, the code updates and saves the output_df DataFrame with the calculated variables obtained from the analysis.

In [ ]:
output_df = pd.read_excel('Output Data Structure.xlsx')

# URL_ID 44 ,57, 144 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table
output_df.drop([44-37,57-37,144-37], axis = 0, inplace=True)

# These are the required parameters
variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
  output_df.iloc[:,i+2] = var

#now save the dataframe to the disk
output_df.to_csv('Output_Data.csv',index=False)